In [1]:
from notebookutils import mssparkutils
from pyspark.sql.functions import * 

StatementMeta(sparkpool, 2, 2, Finished, Available)

In [3]:
sourceaccountname= 'juanprojects'
sourcecontrainer = 'imdb-project'
sourcelinkedservice = 'ls_adls_transform'
sourcefilelocation = 'ingest/'

StatementMeta(sparkpool, 2, 4, Finished, Available)

In [4]:
%%pyspark

source_full_storage_account_name = "juanprojects.dfs.core.windows.net"
spark.conf.set(f"spark.storage.synapse.{source_full_storage_account_name}.linkedServiceName", "ls_adls_transform")
sc._jsc.hadoopConfiguration().set(f"fs.azure.account.oauth.provider.type.{source_full_storage_account_name}", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")

file = f'abfss://{sourcecontrainer}@{sourceaccountname}.dfs.core.windows.net/{sourcefilelocation}'


StatementMeta(sparkpool, 2, 5, Finished, Available)

In [8]:
%%pyspark
# Python code
# spark.conf.set(f"spark.storage.synapse.{sourceaccountname}.linkedServiceName", "ls_adls_transform")
# sc._jsc.hadoopConfiguration().set(f"fs.azure.account.oauth.provider.type.{sourceaccountname}", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")

# df = spark.read.csv('abfss://imdb-project@juanprojects.dfs.core.windows.net/ingest/')

StatementMeta(sparkpool, 2, 9, Finished, Available)

In [6]:
df = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load(file)

StatementMeta(sparkpool, 2, 7, Finished, Available)

In [9]:
df.show()

StatementMeta(sparkpool, 2, 10, Finished, Available)

+--------------------+--------------------+-----------+-------+----------+---------+-------+----------+
|               Title|               Genre|ReleaseDate|Runtime|IMDB Score| Language|  Views| AddedDate|
+--------------------+--------------------+-----------+-------+----------+---------+-------+----------+
|      The Open House|     Horror thriller| 19-01-2018|     94|       3.2|  English|  60545|21-01-2023|
|Searching for Sheela|         Documentary| 22-04-2021|     58|       4.1|  English| 232072|21-01-2023|
|The Last Thing He...|  Political thriller| 21-02-2020|    115|       4.3|  English| 150823|21-01-2023|
|       Thunder Force|    Superhero-Comedy| 09-04-2021|    105|       4.4|  English| 127553|21-01-2023|
|I Am the Pretty T...|              Horror| 28-10-2016|     89|       4.6|  English| 126030|21-01-2023|
|                  IO|Science fiction/D...| 18-01-2019|     95|       4.7|  English| 122853|21-01-2023|
|  Mrs. Serial Killer|            Thriller| 01-05-2020|    106| 

##### Duplicate Rows

In [10]:
df.groupBy(df.columns).count().show()

StatementMeta(sparkpool, 2, 11, Finished, Available)

+--------------------+--------------------+-----------+-------+----------+---------------+------+----------+-----+
|               Title|               Genre|ReleaseDate|Runtime|IMDB Score|       Language| Views| AddedDate|count|
+--------------------+--------------------+-----------+-------+----------+---------------+------+----------+-----+
| Love Wedding Repeat|     Romantic comedy| 10-04-2020|    100|       5.5|        English|590595|15-01-2023|    1|
|      A 3 Minute Hug|         Documentary| 28-10-2019|     28|       6.5|English/Spanish| 95960|21-01-2023|    1|
|               Benji|         Family film| 16-03-2018|     87|       6.3|        English| 75137|15-01-2023|    1|
|             Mascots|        Mockumentary| 13-10-2016|     95|       5.8|        English|451894|15-01-2023|    1|
|        Wine Country|              Comedy| 10-05-2019|    103|       5.5|        English|198243|15-01-2023|    1|
|The Woman in the ...|Psychological thr...| 14-05-2021|    100|       5.7|      

In [12]:
df.groupBy(df.columns).count().filter(col("count")>1).show()

StatementMeta(sparkpool, 2, 13, Finished, Available)

+--------------------+-------------------+-----------+-------+----------+--------+------+----------+-----+
|               Title|              Genre|ReleaseDate|Runtime|IMDB Score|Language| Views| AddedDate|count|
+--------------------+-------------------+-----------+-------+----------+--------+------+----------+-----+
|         Point Blank|             Action| 12-07-2019|     86|       5.7| English|243230|21-01-2023|    2|
|    Caught by a Wave|Romantic teen drama| 25-03-2021|     99|       5.7| Italian|246360|21-01-2023|    2|
|            Good Sam|              Drama| 16-05-2019|     89|       5.7| English|442082|21-01-2023|    2|
|   When We First Met|    Romantic comedy| 09-02-2018|     97|       6.4| English|300803|21-01-2023|    2|
|        Squared Love|    Romantic comedy| 11-02-2021|    102|       5.0|  Polish|  3739|15-01-2023|    2|
|The Most Assassin...|           Thriller| 07-09-2018|    102|       5.3|  French|162122|02-02-2023|    2|
|                null|               

In [13]:
df_nodups= df
df_nodups = df_nodups.drop_duplicates()

StatementMeta(sparkpool, 2, 14, Finished, Available)

In [16]:
df_nodups.groupBy(df_nodups.columns).count().filter(col("count")>1).show()

StatementMeta(sparkpool, 2, 17, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+-----+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|count|
+-----+-----+-----------+-------+----------+--------+-----+---------+-----+
+-----+-----+-----------+-------+----------+--------+-----+---------+-----+



##### Null Values

In [17]:
#-- checking for NULL values
df_nodups.filter(df_nodups.Title.isNull()).show()

StatementMeta(sparkpool, 2, 18, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
| null| null|       null|   null|      null|    null| null|     null|
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [18]:
df_nodups.select(\
    [count(when(isnan(c)| col(c).isNull(), c)).alias(c) for c in df_nodups.columns]
    ).show()

StatementMeta(sparkpool, 2, 19, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
|    1|    2|          1|      1|         1|      17|    1|        1|
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [21]:
df_nodups = df_nodups.na.drop(how="all")

StatementMeta(sparkpool, 2, 22, Finished, Available)

In [43]:
#-- replace null values

df_nonulls =  df_nodups.na.fill(value="Unkonwn")

StatementMeta(sparkpool, 2, 44, Finished, Available)

In [39]:
df_nonulls.filter(col('Language').isNull()).show()

StatementMeta(sparkpool, 2, 40, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [25]:
df_nonulls.filter(col('Genre').isNull()).show()

StatementMeta(sparkpool, 2, 26, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [40]:
df_nonulls.filter(col('Genre')=="Unknown").show()
df_nonulls.filter(col('Language')=="Unknown").show()

StatementMeta(sparkpool, 2, 41, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
+-----+-----+-----------+-------+----------+--------+-----+---------+

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [44]:
#Create new column based on IMdb ranking
df_nonulls = df_nonulls.withColumn("IMDB Category",
    when(col("IMDB Score").between(1, 2.9), "Very Low")
    .when(col("IMDB Score").between(3, 4.9), "Low")
    .when(col("IMDB Score").between(5, 7.9), "Medium")
    .otherwise("High")
)

StatementMeta(sparkpool, 2, 45, Finished, Available)

In [45]:
display(df_nonulls)

StatementMeta(sparkpool, 2, 46, Finished, Available)

SynapseWidget(Synapse.DataFrame, 89e94b81-996a-4995-90ae-ef3429dd3c96)

In [46]:
# new column in hours

df = df_nonulls.withColumn("RuntimeinHours", round(df_nonulls["Runtime"] / 60, 2))


StatementMeta(sparkpool, 2, 47, Finished, Available)

In [47]:
display(df)
df = df.drop("Runtime")

StatementMeta(sparkpool, 2, 48, Finished, Available)

SynapseWidget(Synapse.DataFrame, 51f84367-0243-4d2d-a0bf-fee4daa88361)

In [48]:
df = df.withColumn("Runtime_Category",\

           when(col("RuntimeInHours").between(0,1.30),"Short Runtime")\

           .when(col("RuntimeInHours").between(1.31,2.15) , "Medium Runtime")\

           .otherwise("LongRuntime"))

StatementMeta(sparkpool, 2, 49, Finished, Available)

In [54]:
df.printSchema()

StatementMeta(sparkpool, 2, 55, Finished, Available)

root
 |-- Title: string (nullable = false)
 |-- Genre: string (nullable = false)
 |-- ReleaseDate: string (nullable = false)
 |-- IMDB_Score: double (nullable = true)
 |-- Language: string (nullable = false)
 |-- Views: integer (nullable = true)
 |-- AddedDate: string (nullable = false)
 |-- IMDB_Category: string (nullable = false)
 |-- RuntimeinHours: double (nullable = true)
 |-- Runtime_Category: string (nullable = false)



In [55]:
## Change string to Date

df = df.withColumn("ReleaseDate", to_date("ReleaseDate",'dd-MM-yyyy'))

StatementMeta(sparkpool, 2, 56, Finished, Available)

In [56]:
df = df.withColumnRenamed("IMDB Score", "IMDB_Score")
df = df.withColumnRenamed("IMDB Category", "IMDB_Category")

StatementMeta(sparkpool, 2, 57, Finished, Available)

In [57]:
display(df)

StatementMeta(sparkpool, 2, 58, Finished, Available)

SynapseWidget(Synapse.DataFrame, ca0df82d-22c1-4d60-82b7-291dcb5075b3)

In [64]:
destaccountName = 'juanprojects'
destcontainer = 'imdb-project'
destinationFolder = 'refined/data/'

StatementMeta(sparkpool, 2, 65, Finished, Available)

In [65]:
#Writiing data to ADSL

destpath = f'abfss://{destcontainer}@{destaccountName}.dfs.core.windows.net/{destinationFolder}'
df.write.mode("overwrite").format("parquet").option("header","true").option("inferschema", "true").save(destpath)

StatementMeta(sparkpool, 2, 66, Finished, Available)